In [1]:
import re
import sys
import datetime
sys.path.append(r"D:\往期\QHJ\ZERO")
from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *



# 测试
# 测试
# conn = redis.Redis(**REDIS_TEST_DIC)


conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

In [23]:
data = pd.read_excel(r"C:\Users\qhj01\Desktop\趣领对接服务群运单号（12-12）.xls.xlsx")

In [6]:
headers = {
    "accept": "*/*",
"accept-encoding": "gzip, deflate, br",
"accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
"cache-control": "no-cache",
# "content-length": "53",
"content-type": "application/x-www-form-urlencoded; charset=UTF-8",
"cookie": "PHPSESSID=7gbscq3f8rls83t68ajtgbf2hs",
"origin": "https://app0001.yrapps.cn",
"pragma": "no-cache",
"referer": "https://app0001.yrapps.cn/admin/order/order_delivery/order_id/7801733718286446.html",
"sec-fetch-mode": "cors",
"sec-fetch-site": "same-origin",
"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
"x-requested-with": "XMLHttpRequest",
}

url = r"https://app0001.yrapps.cn/admin/order/send_order.html"



In [4]:
data.columns

Index(['订单号', '商品ID', '商品名', '数量', '规格', '单位', '收件人', '收货地址', '联系方式', '单号',
       '快递 '],
      dtype='object')

In [7]:
for i,row in data.iterrows():

    data = {
        "express_info": row['快递 '],
        "express_id": row['单号'],
        "order_id": row['订单号']
    }
    print(data)
    requests.post(url,headers=headers,data=data)
    
#     requests.post()
    
    
    

{'express_info': '中通', 'express_id': 75319805628524, 'order_id': 7801733718286446}
{'express_info': '中通', 'express_id': 75319805628565, 'order_id': 19121251101531}
{'express_info': '中通', 'express_id': 75319805773360, 'order_id': 19121248494810}
{'express_info': '中通', 'express_id': 75319805628616, 'order_id': 19121253985148}
{'express_info': '中通', 'express_id': 75319805628537, 'order_id': 19121249485597}
{'express_info': '中通', 'express_id': 75319805628578, 'order_id': 19121252575210}
{'express_info': '中通', 'express_id': 75319805923946, 'order_id': 19121257485710}
{'express_info': '中通', 'express_id': 75319805773344, 'order_id': 19121298101101}
{'express_info': '中通', 'express_id': 75319805923933, 'order_id': 19121210253569}
{'express_info': '中通', 'express_id': 75319805628552, 'order_id': 19121297579856}
{'express_info': '中通', 'express_id': 75319805628593, 'order_id': 19121298574910}
{'express_info': '中通', 'express_id': 75319805923959, 'order_id': 19121297101535}
{'express_info': '中通', 'ex

In [16]:
data


,订单号,商品ID,商品名,数量,规格,单位,收件人,收货地址,联系方式,单号,快递
0,7801733718286446,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,郑丽君,湖南省长沙市天心区桂花坪街道园艺路兴汝金城2期14栋2503,13786171008,75319805628524,中通
1,19121251101531,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,张雅淇,湖南省长沙市岳麓区望兴西西里13栋小卖部,15576667890,75319805628565,中通
2,19121248494810,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,于菲菲,湖南省长沙市开福区汐园庭院餐厅,18007316578,75319805773360,中通
3,19121253985148,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,黎雅杰,湖南省长沙市芙蓉区望龙小区A25栋三单元501号,13975187591,75319805628616,中通
4,19121249485597,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,李小姐,湖南省长沙市天心区回龙山巷回龙公寓b栋1002,18007483266,75319805628537,中通
5,19121252575210,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,彭松梅,湖南省长沙市岳麓区中海国际社区4期,18684819641,75319805628578,中通
6,19121257485710,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,万威,湖南省湘潭市湘乡市文华苑1栋401,18975281371,75319805923946,中通
7,19121298101101,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,祁鑫曦,湖南省长沙市开福区民主东街11号湖南省总工会,13187212768,75319805773344,中通
8,19121210253569,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,刘娟,湖南省湘潭市湘乡市望春门街道新汽车站联盟安置区联盟实惠超市旁边,17711627602,75319805923933,中通
9,19121297579856,S0003P0001C0001P01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,老气球,湖南省长沙市雨花区汇财中心(107国道东)恒丰银行(长沙分行),18173191016,75319805628552,中通


In [24]:
for i,row in data.iterrows():
    spp_id = row['商品ID'][:5]
    print(row['单号'])
    r = conn.smove("ORDER:NEW:%s" %(spp_id),
               "ORDER:RC:%s" %(spp_id)
               ,row['订单号'])
    print(r)

75319805628524
True
75319805628565
True
75319805773360
True
75319805628616
True
75319805628537
True
75319805628578
True
75319805923946
True
75319805773344
True
75319805923933
True
75319805628552
True
75319805628593
True
75319805923959
True
75319805773316
True
75319805773372
True
75319805923961
True
75319805773329
True
75319805628540
True
75319805773316
True
75319805628586
True
75319805773357
True
75319805773331
True
75319805628603
True


In [3]:
data = pd.read_excel(r"D:\奇货居\work\外发订单\新订单\三金_新订单 2019-12-13 15_38_09.xlsx",index=False)

In [5]:
conn.sadd("order:old",*data['订单号'].to_list())

20